# Recurrent Capsule Network for IMDB Reviews Dataset 

## Load Train and Test Data

In [0]:
!wget https://raw.githubusercontent.com/ClairBee/cs909/master/reutersCSV.csv


Redirecting output to ‘wget-log.3’.


In [0]:
train_df = pd.read_csv('reutersCSV.csv', encoding='ISO-8859-1')
train_df.head()

,pid,fileName,purpose,topic.acq,topic.alum,topic.austdlr,topic.austral,topic.barley,topic.bfr,topic.bop,...,topic.tung,topic.tung.oil,topic.veg.oil,topic.wheat,topic.wool,topic.wpi,topic.yen,topic.zinc,doc.title,doc.text
0,1,0,train,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,BAHIA COCOA REVIEW,Showers continued throughout the week in the B...
1,2,0,train,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,STANDARD OIL &lt;SRD> TO FORM FINANCIAL UNIT,Standard Oil Co and BP North America Inc said ...
2,3,0,train,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,TEXAS COMMERCE BANCSHARES &lt;TCB> FILES PLAN,Texas Commerce Bancshares Inc's Texas Commerce...
3,4,0,train,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,TALKING POINT/BANKAMERICA &lt;BAC> EQUITY OFFER,BankAmerica Corp is not under pressure to act ...
4,5,0,train,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE,The U.S. Agriculture Department reported the f...


In [0]:
train_df.shape

(21578, 140)

In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2018-10-19 08:10:33--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  20.1MB/s    in 6.6s    

2018-10-19 08:10:39 (12.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [1]:
import tarfile

tar = tarfile.open('/content/reuters21578.tar.gz', "r")
tar.extractall()
tar.close()

In [0]:
import tarfile

tar = tarfile.open('/content/aclImdb_v1.tar.gz', "r")
tar.extractall()
tar.close()

In [0]:
import os
os.listdir('/content/')

['.config',
 'sample_data',
 'adc.json',
 'aclImdb_v1.tar.gz',
 'ag_news_csv',
 'yelp_academic_dataset_review.json',
 'aclImdb']

In [0]:
len(os.listdir('/content/aclImdb/train/neg'))

12500

In [0]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
def load_all(dir_path):
    seq = []
    for filename in os.listdir(dir_path):
    if not filename.endswith(".txt"):
        continue
    path = dir_path + '/' + filename
    doc = load_doc(path)
    seq.append(doc)
    return seq

In [0]:
import pandas as pd

train_neg = pd.DataFrame(load_all('/content/aclImdb/train/neg'))
train_neg.columns = ['text']
train_neg['output'] = 0
train_neg.head()

,text,output
0,Another case of a decent DVD case betraying th...,0
1,"Two days after seeing this thing, I'm still in...",0
2,"It doesn't even merit a review, other than as ...",0
3,This movie should be nominated for a new genre...,0
4,"The original ""Vanishing Point"" was a great fli...",0


In [0]:
train_neg.loc[19]['text']

'To say this film stinks would be insulting to skunks. As the other commenter says, this movie is insulting to anyone over the mental age of 7 (it is especially, incredibly insulting to gays). It is awful - and not in a "so bad it\'s funny" sort of way either - it\'s just plain awful. No, I have to say it: IT STINKS! (sorry skunks).<br /><br />From the opening credits to the end titles there is hardly more than 10 seconds of this movie worth opening your eyes for. The "plot" is incoherent, the characterization non-existent, the acting is of the over the top mugging "look at me I\'m being funny!" school and so it goes on. The set pieces are clumsily set up (if at all) and are badly executed, it\'s just awful on every front - apart from the music maybe, I don\'t remember thinking the music stinks (apart from the songs).<br /><br />To be fair to the makers, they lay their cards on the table pretty quickly: the opening credits include the title "Also starring Ertha Kitt as the voice of Bet

In [0]:
train_pos = pd.DataFrame(load_all('/content/aclImdb/train/pos'))
train_pos.columns = ['text']
train_pos['output'] = 1

test_neg = pd.DataFrame(load_all('/content/aclImdb/test/neg'))
test_neg.columns = ['text']
test_neg['output'] = 0

test_pos = pd.DataFrame(load_all('/content/aclImdb/test/pos'))
test_pos.columns = ['text']
test_pos['output'] = 1
test_pos.head()

,text,output
0,"I absolutely loved this series, and was very s...",1
1,Bottom has been my favourite sitcom ever since...,1
2,Cheers to all the wonderful fans of this film ...,1
3,Woaww Is it only now that you notice the links...,1
4,"The English are a little too evil, the Scots a...",1


In [0]:
train_df = pd.concat([train_neg, train_pos])
test_df = pd.concat([test_neg, test_pos])
train_df.shape, test_df.shape

((25000, 2), (25000, 2))

## Shuffle Data

In [0]:
del train_pos, test_pos, train_neg, test_neg

In [0]:
from sklearn.utils import shuffle
train_df = shuffle(train_df)
print (train_df.shape)
train_df.reset_index(inplace=True)
train_df.head()

(25000, 2)


,index,text,output
0,15365,There are few uplifting things to say about th...,0
1,9188,Now I had the best intentions when watching th...,0
2,12985,This stylistically sophisticated visual game p...,1
3,6691,An adaption of the book 'Finding Fish'. This s...,1
4,17384,"In the Comic, Modesty is strong. Alexandra Sta...",0


In [0]:
train_df.drop('index', axis=1, inplace=True)
train_df.head()

,text,output
0,There are few uplifting things to say about th...,0
1,Now I had the best intentions when watching th...,0
2,This stylistically sophisticated visual game p...,1
3,An adaption of the book 'Finding Fish'. This s...,1
4,"In the Comic, Modesty is strong. Alexandra Sta...",0


In [0]:
test_df = shuffle(test_df)
print (test_df.shape)
test_df.reset_index(inplace=True)
test_df.drop('index', axis=1, inplace=True)
test_df.head()

(25000, 2)


,text,output
0,"There are times when, less than halfway throug...",0
1,"What a silly movie. While it looks nice, it do...",0
2,I have seen this show when I was younger. It i...,1
3,I LOVED this movie because Bobbie Phillips can...,1
4,Although the figures are higher in proportion ...,0


In [0]:
os.listdir('/content/')

['.config',
 'sample_data',
 'aclImdb_v1.tar.gz',
 'wget-log',
 'wget-log.1',
 'cap_yelp_p.h5',
 'glove.840B.300d.zip',
 'adc.json',
 'yelp_review_polarity_csv',
 'pred_2out_classes.csv',
 'aclImdb',
 'datalab']

In [0]:
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, model_selection, metrics
import warnings
warnings.filterwarnings('ignore')
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.optimizers import Adam

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

%matplotlib inline

Using TensorFlow backend.


In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints

EMBEDDING_FILE='/content/datalab/glove.840B.300d.txt'

MAX_SEQUENCE_LENGTH = 300
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = 300
num_dense = 256
rate_drop_lstm = 0.2
rate_drop_dense = 0.2

act = 'relu'

In [0]:
def cleanData(text, stemming = False, lemmatize=False):    
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in text.split()])
    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w) for w in text.split()])
    return text

## Download Glove

In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

import zipfile
zip_ref = zipfile.ZipFile('glove.840B.300d.zip', 'r')
zip_ref.extractall('/content/datalab/')
zip_ref.close()

## Preprocessing

In [0]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.layers import concatenate, CuDNNGRU
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys

In [0]:
import re

col_text = 'text'
list_classes = ['output']

print('Processing text dataset')

train_df[col_text] = train_df[col_text].map(lambda x: cleanData(x, stemming=False, lemmatize=False))
test_df[col_text] = test_df[col_text].map(lambda x: cleanData(x, stemming=False, lemmatize=False))

#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)
#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    text=special_character_removal.sub('',text)
    #Replace Numbers
    text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    return(text)


list_sentences_train = train_df[col_text].fillna("NA").values
y = train_df[list_classes].values
list_sentences_test = test_df[col_text].fillna("NA").values


comments = []
for text in list_sentences_train:
    comments.append(text_to_wordlist(text))
    
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text))

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(comments + test_comments)

sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

Processing text dataset
Found 100461 unique tokens
Shape of data tensor: (25000, 300)
Shape of label tensor: (25000, 1)
Shape of test_data tensor: (25000, 300)


In [0]:
data_post = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH,padding='post', truncating='post')
print('Shape of data tensor:', data_post.shape)
print('Shape of label tensor:', y.shape)

test_data_post = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
print('Shape of test_data tensor:', test_data_post.shape)

Shape of data tensor: (25000, 300)
Shape of label tensor: (25000, 1)
Shape of test_data tensor: (25000, 300)


In [0]:
print('Indexing word vectors')

count = 0
embeddings_index = {}
f = open(EMBEDDING_FILE)
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    embeddings_index[word] = coefs.reshape(-1)
    coef = embeddings_index[word]
f.close()

print('Found %d word vectors of glove.' % len(embeddings_index))
emb_mean,emb_std = coef.mean(), coef.std()
print(emb_mean,emb_std)

print('Total %s word vectors.' % len(embeddings_index))


Indexing word vectors
Found 2195895 word vectors of glove.
-0.01444638 0.47249147
Total 2195895 word vectors.


In [0]:
# MAX_NB_WORDS = 87733

In [0]:
print('Preparing embedding matrix')
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 22596


In [0]:
max_features=MAX_NB_WORDS
maxlen=300
embed_size=300

## Model

In [0]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [0]:
from keras.layers import K, Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D
gru_len = 128
Routings = 5
Num_capsule = 10
Dim_capsule = 16
dropout_p = 0.25
rate_drop_dense = 0.28

def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale


# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(K.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = K.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)


In [0]:
import sys
!{sys.executable} -m pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.2)


In [0]:
def get_model():
    inp1 = Input(shape=(300,))
    embed1 = Embedding(max_features, 300, input_length=300, weights=[embedding_matrix], trainable=False)(inp1)
    embed1 = SpatialDropout1D(0.4)(embed1)
    
    x1 = Bidirectional(CuDNNGRU(100, return_sequences=True))(embed1)
    capsule1 = Capsule(num_capsule=10, dim_capsule=16, routings=5,share_weights=True)(x1)
    # output_capsule1 = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule1)
    capsule1 = GlobalMaxPooling1D()(capsule1)
    
    inp2 = Input(shape=(300,))
    embed2 = Embedding(max_features, 300, input_length=300, weights=[embedding_matrix], trainable=False)(inp2)
    embed2 = SpatialDropout1D(0.4)(embed2)
    
    x2 = Bidirectional(CuDNNGRU(100, return_sequences=True))(embed2)
    capsule2 = Capsule(num_capsule=10, dim_capsule=16, routings=5,share_weights=True)(x2)
    # output_capsule2 = Lambda(lambda x: K.sqrt(K.sum(K.square(x), 2)))(capsule2)
    capsule2 = GlobalMaxPooling1D()(capsule2)
    
    capsule = concatenate([capsule1, capsule2])
    capsule = Dropout(0.2)(capsule)
    output = Dense(1, activation='sigmoid')(capsule)
    
    model = Model(inputs=[inp1, inp2], outputs=output)
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=1e-3,decay=0),
        metrics=['accuracy'])
#     model.summary()
    return model

## Train the model

In [0]:
file_path="cap_imdb.h5"
model = get_model()
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
hist = model.fit([data, data_post], y, batch_size=128, epochs=15, verbose=1, validation_split=0.075, shuffle=True, callbacks = [check_point, early_stop])
model.load_weights(file_path)
best_score = min(hist.history['val_loss'])

Train on 23125 samples, validate on 1875 samples
Epoch 1/15
23125/23125 [==============================] - 63s 3ms/step - loss: 0.4708 - acc: 0.7737 - val_loss: 0.3375 - val_acc: 0.8539

Epoch 00001: val_loss improved from inf to 0.33752, saving model to cap_imdb.h5
Epoch 2/15
23125/23125 [==============================] - 60s 3ms/step - loss: 0.3470 - acc: 0.8541 - val_loss: 0.3015 - val_acc: 0.8773

Epoch 00002: val_loss improved from 0.33752 to 0.30153, saving model to cap_imdb.h5
Epoch 3/15
23125/23125 [==============================] - 60s 3ms/step - loss: 0.3216 - acc: 0.8686 - val_loss: 0.2894 - val_acc: 0.8800

Epoch 00003: val_loss improved from 0.30153 to 0.28937, saving model to cap_imdb.h5
Epoch 4/15
23125/23125 [==============================] - 60s 3ms/step - loss: 0.2967 - acc: 0.8782 - val_loss: 0.2743 - val_acc: 0.8789

Epoch 00004: val_loss improved from 0.28937 to 0.27432, saving model to cap_imdb.h5
Epoch 5/15
23125/23125 [==============================] - 60s 3ms/s

In [0]:
model.load_weights(file_path)
min(hist.history['val_loss'])

0.23686041742165884

## Prediction

In [0]:
test_predicts = model.predict([test_data, test_data_post], batch_size=1024, verbose=1)

25000/25000 [==============================] - 13s 537us/step


In [0]:
pred = pd.DataFrame(test_predicts)
pred.columns = ['output']
pred.head()

,output
0,0.006587
1,0.169970
2,0.991626
3,0.992399
4,0.272858


In [0]:
model.evaluate([test_data, test_data_post], test_df['output'], batch_size=1024, verbose=1)

25000/25000 [==============================] - 13s 519us/step


[0.22299199876785278, 0.9094399995613098]

In [0]:
pred.describe()

,output
count,25000.000000
mean,0.514638
std,0.434968
min,0.004995
25%,0.024055
50%,0.578497
75%,0.979142
max,0.996933


### Convert to One Hot 

In [0]:
pred['output'] = pred['output'].map(lambda x: 0 if x<0.5 else 1)

## Test Accuracy

In [0]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(test_df['output'], pred['output'])

print ('Mean Accuracy on Test Data: ', acc)

Mean Accuracy on Test Data:  0.90944


In [0]:
from sklearn.metrics import roc_auc_score, precision_score

auc = roc_auc_score(test_df['output'], pred['output'])
pre = precision_score(test_df['output'], pred['output'])

print ('Mean ROC AUC on Test Data: ', auc)
print ('Mean Precision on Test Data: ', pre)

Mean ROC AUC on Test Data:  0.9094400000000001
Mean Precision on Test Data:  0.8971135940409684


In [0]:
# test_df.reset_index(inplace=True)
test_df.tail()

,text,output
24995,amidst all the many problems that make this a ...,0
24996,i was surprised that the makers of this movie ...,0
24997,if you been there and done that you will absol...,1
24998,this is one of the very very few films that ar...,1
24999,it is a nice comedy it has the great features ...,1


In [0]:
pred.tail()

,output
24995,0
24996,0
24997,1
24998,1
24999,1


In [0]:
test_df.drop('index', inplace=True, axis=1)
pred.shape, test_df.shape

((25000, 1), (25000, 2))

## BiGRU Model

In [0]:
from keras.optimizers import Adam

adam_opt = Adam(lr=1e-3, decay=0.001)

def get_gru_model():
    comment_input = Input(shape=(MAX_SEQUENCE_LENGTH,))
    comment_input_post = Input(shape=(MAX_SEQUENCE_LENGTH,))

    x1 = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], 
                  input_length=MAX_SEQUENCE_LENGTH)(comment_input)
    x1 = SpatialDropout1D(0.4)(x1)
    x1 = Bidirectional(CuDNNGRU(80, return_sequences=True))(x1)
    avg_pool1 = GlobalAveragePooling1D()(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    
#     x2 = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], 
#     x2 = SpatialDropout1D(0.4)(x2)
#     x2 = Bidirectional(CuDNNGRU(80, return_sequences=True))(x2)
#     avg_pool2 = GlobalAveragePooling1D()(x2)
#     max_pool2 = GlobalMaxPooling1D()(x2)

#     conc = concatenate([avg_pool1, max_pool1, avg_pool2, max_pool2])
    conc = concatenate([avg_pool1, max_pool1])
    conc = Dropout(0.2)(conc)
    preds = Dense(1, activation="sigmoid")(conc)
#     model = Model(inputs=[comment_input, comment_input_post], outputs=preds)
    model = Model(inputs=[comment_input], outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    return model

In [0]:
file_path="gru_imdb.h5"
model = get_gru_model()
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, save_best_only = True, mode = "min")
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
hist = model.fit([data], y, batch_size=128, epochs=5, verbose=1, validation_split=0.075, shuffle=True, callbacks = [check_point, early_stop])
model.load_weights(file_path)
best_score = min(hist.history['val_loss'])

Train on 23125 samples, validate on 1875 samples
Epoch 1/15
23125/23125 [==============================] - 62s 3ms/step - loss: 0.4478 - acc: 0.7820 - val_loss: 0.3126 - val_acc: 0.8619

Epoch 00001: val_loss improved from inf to 0.31263, saving model to gru_imdb.h5
Epoch 2/15
23125/23125 [==============================] - 60s 3ms/step - loss: 0.2627 - acc: 0.8953 - val_loss: 0.2682 - val_acc: 0.8928

Epoch 00002: val_loss improved from 0.31263 to 0.26821, saving model to gru_imdb.h5
Epoch 3/15
23125/23125 [==============================] - 59s 3ms/step - loss: 0.1911 - acc: 0.9273 - val_loss: 0.3228 - val_acc: 0.8837

Epoch 00003: val_loss did not improve from 0.26821
Epoch 4/15
23125/23125 [==============================] - 59s 3ms/step - loss: 0.1372 - acc: 0.9504 - val_loss: 0.2704 - val_acc: 0.8960

Epoch 00004: val_loss did not improve from 0.26821
Epoch 5/15
23125/23125 [==============================] - 59s 3ms/step - loss: 0.0955 - acc: 0.9671 - val_loss: 0.3190 - val_acc: 0.8

KeyboardInterrupt: ignored

In [0]:
print (file_path)
model.load_weights(file_path)
min(hist.history['val_loss'])

gru_imdb.h5


0.23686041742165884

In [0]:
test_predicts = model.predict([test_data, test_data_post], batch_size=1024, verbose=1)
pred = pd.DataFrame(test_predicts)
pred.columns = ['output']
pred.head()

25000/25000 [==============================] - 8s 332us/step


,output
0,0.010634
1,0.043800
2,0.996941
3,0.985260
4,0.041864


In [0]:
pred.describe()

,output
count,25000.000000
mean,0.505206
std,0.434051
min,0.000047
25%,0.024188
50%,0.516205
75%,0.979992
max,0.999987


In [0]:
model.evaluate([test_data, test_data_post], test_df['output'], batch_size=1024, verbose=1)

25000/25000 [==============================] - 8s 324us/step


[0.25016355454444883, 0.8999599995613098]

In [0]:
pred['output'] = pred['output'].map(lambda x: 0 if x<0.5 else 1)

In [0]:
acc = accuracy_score(test_df['output'], pred['output'])

print ('Mean Accuracy on Test Data: ', acc)

Mean Accuracy on Test Data:  0.89996
